In [1]:
!pip install pandas 
!pip install numpy 
!pip install spacy
!pip install -U scikit-learn
!pip install seaborn
!pip install transformers

/usr/bin/zsh: /home/roberto/.conda/envs/tensorflow-env/lib/libncursesw.so.6: no version information available (required by /usr/bin/zsh)
/usr/bin/zsh: /home/roberto/.conda/envs/tensorflow-env/lib/libncursesw.so.6: no version information available (required by /usr/bin/zsh)
/usr/bin/zsh: /home/roberto/.conda/envs/tensorflow-env/lib/libncursesw.so.6: no version information available (required by /usr/bin/zsh)
/usr/bin/zsh: /home/roberto/.conda/envs/tensorflow-env/lib/libncursesw.so.6: no version information available (required by /usr/bin/zsh)
/usr/bin/zsh: /home/roberto/.conda/envs/tensorflow-env/lib/libncursesw.so.6: no version information available (required by /usr/bin/zsh)
/usr/bin/zsh: /home/roberto/.conda/envs/tensorflow-env/lib/libncursesw.so.6: no version information available (required by /usr/bin/zsh)


In [2]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from utils.emfd import *
from utils.spacy_preprocessor import SpaCyPreProcessor
import pickle
from tqdm.notebook import tqdm

2023-06-24 11:47:46.922480: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-24 11:47:47.551179: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-24 11:47:50.209439: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-24 11:47:50.213985: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [3]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
# text = "Good night 😊"
# text = preprocess(text)
# encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)
# scores = output[0][0].detach().numpy()
# scores = softmax(scores)

# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)

# text = "Good night 😊"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

In [4]:
#save pre trained model

model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained("models/" + MODEL)

2023-06-24 11:47:52.441808: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-24 11:47:52.445194: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-24 11:47:52.451005: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [5]:
#load model

model = TFAutoModelForSequenceClassification.from_pretrained("models/" + MODEL)

2023-06-24 11:47:55.819532: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 154414080 exceeds 10% of free system memory.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at models/cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [6]:
mfrc = pd.read_csv("datasets/edited/mfrc_cleaned.csv")

In [7]:
def get_scores(scores):
    score = dict()
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    for i in range(scores.shape[0]):
        l = labels[ranking[i]]
        s = scores[ranking[i]]
        score[l] = np.round(float(s), 4)

    return score


mfrc["positive"] = 0
mfrc["neutral"] = 0
mfrc["negative"] = 0


for i in tqdm(mfrc.index):
    text = mfrc["cleaned_text"][i]
    encoded_input = tokenizer(text, return_tensors='tf')
    output = model(**encoded_input)
    scores = output[0][0].numpy()
    scores = softmax(scores)
    dict_scores = get_scores(scores)
    mfrc["positive"][i] = dict_scores["positive"]
    mfrc["neutral"][i] = dict_scores["neutral"]
    mfrc["negative"][i] = dict_scores["negative"]

  0%|          | 0/61226 [00:00<?, ?it/s]

/tmp/ipykernel_12429/3412493262.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mfrc["positive"][i] = dict_scores["positive"]
/tmp/ipykernel_12429/3412493262.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mfrc["neutral"][i] = dict_scores["neutral"]
/tmp/ipykernel_12429/3412493262.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mfrc["negative"][i] = dict_scores["negative"]


In [8]:
mfrc

,text,subreddit,bucket,annotator,annotation,confidence,care.virtue,care.vice,fairness.virtue,fairness.vice,loyalty.virtue,loyalty.vice,authority.virtue,authority.vice,sanctity.virtue,sanctity.vice,cleaned_text,positive,neutral,negative
0,That particular part of the debate is especial...,europe,French politics,annotator03,Non-Moral,Confident,0,0,0,0,0,0,0,0,0,0,particular debate especially funny macron expl...,0.0858,0.6045,0.3097
1,That particular part of the debate is especial...,europe,French politics,annotator01,Purity,Confident,0,0,0,0,0,0,0,0,0,0,particular debate especially funny macron expl...,0.0858,0.6045,0.3097
2,That particular part of the debate is especial...,europe,French politics,annotator02,Thin Morality,Confident,0,0,0,0,0,0,0,0,0,0,particular debate especially funny macron expl...,0.0858,0.6045,0.3097
3,"/r/france is pretty lively, with it's own ling...",europe,French politics,annotator03,Non-Moral,Confident,0,0,0,0,0,0,0,0,0,0,r france pretty lively lingo usually delibera...,0.1293,0.4153,0.4554
4,"/r/france is pretty lively, with it's own ling...",europe,French politics,annotator00,Non-Moral,Somewhat Confident,0,0,0,0,0,0,0,0,0,0,r france pretty lively lingo usually delibera...,0.1293,0.4153,0.4554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61221,Well I can discern from your vehemence toward ...,AmItheAsshole,Everyday Morality,annotator05,Equality,Confident,0,0,0,0,0,0,0,0,0,0,discern vehemence woman want think equality hu...,0.0237,0.4024,0.5739
61222,Kick! Punch! It's all in the mind. If you wann...,nostalgia,Everyday Morality,annotator05,Thin Morality,Somewhat Confident,0,0,0,0,0,0,0,0,0,0,kick punch mind wanna test sure find thing tea...,0.0605,0.7493,0.1903
61223,Reddit can’t help you this is some seriously t...,confession,Everyday Morality,annotator05,Thin Morality,Confident,0,0,0,0,0,0,0,0,0,0,reddit help seriously traumatic shit coast fuc...,0.0037,0.0203,0.9760
61224,Yes. Disordered eating is insidious. And Rita ...,AmItheAsshole,Everyday Morality,annotator05,Non-Moral,Somewhat Confident,0,0,0,0,0,0,0,0,0,0,yes disordered eating insidious rita need help...,0.0103,0.2772,0.7125


In [ ]:
mfrc.to_csv("datasets/edited/mfrc_scored_with_sent.csv", index = False, header = True)